### 처리
1. 파라미터 설정
1. 매수 후보 종목 선별
1. 포트폴리오 조회
2. 매도 종목 검색 
4. 매수 종목 선정
1. (날짜 다음날 전환)
1. 매도처리
5. 매수처리


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
import os
import sqlite3
import csv

%matplotlib notebook

In [2]:
# 전일영업일자 계산
def getBeforeWorkingDate(today):
    sql = "SELECT date FROM holiday_calendar WHERE date < {} AND holiday = 0".format(today)
    return pd.read_sql(sql, con, index_col=None).max()['date']

In [3]:
# *****************************
# 파라미터 설정
# *****************************
# 시작날짜
START_DAY = "20090312"
# 종료날짜
END_DAY = "20100106"
# 선별거래종목수
CANDIDATE_STOCK_CNT = 10
# 매수/매도 거래량
AMOUNT_BUY = 10
AMOUNT_SELL = 10
# 거래 주기 일자
CYCLE_DAY = 1



In [4]:
# db file
dbDir = 'C:/Users/jseung21/Documents/CybosPlus/sqlite3'
# DBCP
con = sqlite3.connect(dbDir+"/MondayLab.db")
sqlite3.Connection
# cursor = con.cursor()

sqlite3.Connection

In [ ]:
# 날짜 설정
today = START_DAY
beforeDay = getBeforeWorkingDate(today)
print("today:[{}], beforDay:[{}]".format(today,beforeDay))

In [20]:
# *****************************
# 매수 후보 종목 선별
# *****************************

# kospi 200 종목 검색
dfKospi200 = pd.read_sql("SELECT * FROM kospi200", con, index_col=None)
    
# kospi200 종목중에서 매수 후보 주식 선별
candidate = []
for stockCode in dfKospi200['stockCode']:
    # 
    sqlTdoay = "SELECT stockCode FROM origin_trade WHERE 1=1 \
                AND stockCode = '{}' \
                AND date = {} \
                AND days5MovAvr > days20MovAvr" \
                .format(stockCode,today)
    sqlBeforeDay = "SELECT stockCode FROM origin_trade WHERE 1=1 \
                AND stockCode = '{}' \
                AND date = {} \
                AND days5MovAvr <= days20MovAvr" \
                .format(stockCode,beforeDay)
    
    result1 = pd.read_sql(sqlBeforeDay, con, index_col=None)
    
    if len(result1) != 0:
        result2 = pd.read_sql(sqlBeforeDay, con, index_col=None)['stockCode'][0]
        candidate.append(result2)
    
print("candidate : [{}]".format(len(candidate)))

# 매수 후보 종목 csv 파일생성
fileName = "candidate_{}.csv".format(today)
with open(fileName, 'w') as csvFile:
    writer = csv.writer(csvFile)
    writer.writerow(candidate)

csvFile.close()

KeyboardInterrupt: 

In [15]:
# *****************************
# 포트폴리오 조회
# *****************************
sqlPortfolio = "SELECT stockCode FROM Portfolio WHERE 1=1 \
                AND trading = 1"
portfolio = pd.read_sql(sqlPortfolio, con, index_col=None)

print("portfolio 건수 : [{}] ".format(len(portfolio)))

portfolio 건수 : [0] 


In [21]:
# *****************************
# 매도 종목 검색
# *****************************
stockSell = []
if len(portfolio) != 0:
    for stcokSellCode in portfolio:
        sqlStockSell = "SELECT stockCode FROM WHERE 1=1 \
                        AND date = '{}' \
                        AND stockCode = '{}' \
                        AND days5MovAvr < days20MovAvr" \
                        .format(today, stockCode['stockCode'][0])
        sellStockCode = pd.read_sql(sqlStockSell, con, index_col=None)['stockCode'][0]
        
        if len(sellStockCode) != 0:
            stockSell.append(sellStockCode)
            
    print("stockSell 건수 : [{}]".format(len(stockSell)))
    

In [ ]:
# *****************************
# 매수 종목 선정
# *****************************


In [ ]:
# *****************************
# (날짜 다음날 전환)
# *****************************

In [ ]:
# *****************************
# 매도 처리
# *****************************
if len(stockSell) != 0:
    for sellStockCode in stockSell:
        sqlSell = "select startPrice from origin_trade where 1=1 \
                    AND "
        
    

In [ ]:
# *****************************
# 매수처리
# *****************************
